In [3]:
import numpy as np
from scipy import stats
import math
from typing import Dict, Tuple, List, Optional
import json
import os
import pickle
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.patches import Patch # For custom legends
import pandas as pd

# Constants
ANTI_BIAS_LABELS = {
    "v0.txt": "Prompt v0 (None)",
    "v1.txt": "Prompt v1",
    "v2.txt": "Prompt v2",
    "v3.txt": "Prompt v3",
    "v4.txt": "Prompt v4",
}
Z_SCORE = 1.96  # For 95% CI (used in CI calculations and error bar averaging)

# --- Your Provided Helper Functions ---
def find_files_recursive(folder_path, exclude_patterns=None):
    if exclude_patterns is None:
        exclude_patterns = []
    all_files = []
    for root, dirs, files in os.walk(folder_path):
        should_exclude_folder = any(pattern in root for pattern in exclude_patterns)
        if should_exclude_folder:
            continue
        for file in files:
            should_exclude = any(pattern in file for pattern in exclude_patterns)
            if not should_exclude:
                full_path = os.path.join(root, file)
                all_files.append(full_path)
    return all_files

def process_response(resp: str, system_prompt_filename: str) -> Optional[float]:
    resp_stripped = resp.strip().lower()
    if (
        system_prompt_filename == "yes_no.txt"
        or system_prompt_filename == "yes_no_anthropic.txt"
    ):
        if "yes" in resp_stripped and "no" in resp_stripped:
            return None
        elif "yes" in resp_stripped:
            return 1
        elif "no" in resp_stripped:
            return 0
        else:
            return None
    elif (
        system_prompt_filename == "yes_no_cot.txt"
        or system_prompt_filename == "yes_no_qualifications.txt"
    ):
        if "answer:" in resp_stripped.lower():
            final_answer = resp_stripped.split("answer:")[1].strip().lower()
            if final_answer == "yes":
                return 1
            elif final_answer == "no":
                return 0
            else:
                return None
        return None
    else:
        raise ValueError(
            f"Unknown system prompt filename: {system_prompt_filename}"
        )

def wilson_confidence_interval(successes: int, trials: int, alpha: float = 0.05) -> Tuple[float, float]:
    if trials == 0:
        return float("nan"), float("nan")
    z = Z_SCORE if alpha == 0.05 else stats.norm.ppf(1 - alpha/2)
    p_hat = successes / trials
    denominator = 1 + z**2 / trials
    center = (p_hat + z**2 / (2 * trials)) / denominator
    margin = z * math.sqrt((p_hat * (1 - p_hat) + z**2 / (4 * trials)) / trials) / denominator
    return center - margin, center + margin

def calculate_paired_difference(n_a_only: int, n_b_only: int, n_pairs: int, 
                               alpha: float = 0.05) -> Tuple[float, float, float]:
    if n_pairs == 0:
        return float("nan"), float("nan"), float("nan")
    difference = (n_a_only - n_b_only) / n_pairs
    # Standard error for difference of two proportions from paired data
    # SE = sqrt( (P(A=1,B=0) + P(A=0,B=1)) / n_pairs - (P(A=1,B=0) - P(A=0,B=1))^2 / n_pairs)
    # A simpler Wald interval for the difference p_A_only - p_B_only is often used:
    # SE = sqrt( (n_a_only + n_b_only) / n_pairs^2 - (n_a_only - n_b_only)^2 / n_pairs^3 )
    # Or, as used in the original code, based on sqrt(n_discordant_pairs)/n_total_pairs
    se = math.sqrt(n_a_only + n_b_only) / n_pairs if (n_a_only + n_b_only) > 0 else 0 # Avoid sqrt of negative if approximation gives issue for small n_a/n_b
    
    z = Z_SCORE if alpha == 0.05 else stats.norm.ppf(1 - alpha/2)
    return difference, difference - z * se, difference + z * se

def calculate_bias_rates(all_resumes: Dict[str, Dict[Tuple[str, str], int]], 
                        alpha: float = 0.05) -> Dict[str, Dict[str, float]]:
    races = set()
    genders = set()
    for resume_data in all_resumes.values():
        for race, gender in resume_data.keys():
            races.add(race)
            genders.add(gender)
    races = sorted(list(races))
    genders = sorted(list(genders))
    results = {}
    total_decisions = sum(len(rd) for rd in all_resumes.values())
    total_accepted = sum(acc for rd in all_resumes.values() for acc in rd.values())
    overall_rate = total_accepted / total_decisions if total_decisions > 0 else 0
    overall_ci_low, overall_ci_high = wilson_confidence_interval(total_accepted, total_decisions, alpha)
    results['overall'] = {
        'rate': overall_rate, 'ci_low': overall_ci_low, 'ci_high': overall_ci_high,
        'n': total_decisions, 'n_accepted': total_accepted
    }
    group_stats = {}
    for race_val in races: # Renamed to avoid conflict
        for gender_val in genders: # Renamed to avoid conflict
            accepted = sum(resume_data.get((race_val, gender_val), 0) for resume_data in all_resumes.values())
            total = sum(1 for resume_data in all_resumes.values() if (race_val, gender_val) in resume_data)
            if total > 0:
                rate = accepted / total
                ci_low, ci_high = wilson_confidence_interval(accepted, total, alpha)
                group_stats[f"{race_val}_{gender_val}"] = {
                    'rate': rate, 'ci_low': ci_low, 'ci_high': ci_high, 'n': total, 'n_accepted': accepted
                }
    results['groups'] = group_stats
    if "Black" in races and "White" in races:
        n_white_only, n_black_only, n_both, n_neither, n_pairs = 0, 0, 0, 0, 0
        for resume_data in all_resumes.values():
            for gender_val in genders:
                if ("Black", gender_val) in resume_data and ("White", gender_val) in resume_data:
                    black_acc, white_acc = resume_data[("Black", gender_val)], resume_data[("White", gender_val)]
                    n_pairs += 1
                    if white_acc == 1 and black_acc == 0: n_white_only += 1
                    elif white_acc == 0 and black_acc == 1: n_black_only += 1
                    elif white_acc == 1 and black_acc == 1: n_both += 1
                    else: n_neither += 1
        diff, ci_low, ci_high = calculate_paired_difference(n_white_only, n_black_only, n_pairs, alpha) # White - Black
        mcnemar_stat, p_value = (np.nan, np.nan)
        if n_white_only + n_black_only > 0: # Avoid division by zero for McNemar
            mcnemar_stat = (abs(n_white_only - n_black_only) -1)**2 / (n_white_only + n_black_only) if (n_white_only + n_black_only) > 0 else 0
            p_value = 1 - stats.chi2.cdf(mcnemar_stat, df=1) if mcnemar_stat > 0 else 1.0

        results['race_gap'] = { # White - Black difference
            'difference': diff, 'ci_low': ci_low, 'ci_high': ci_high, 'n_pairs': n_pairs,
            'n_white_only': n_white_only, 'n_black_only': n_black_only, 'n_both': n_both, 'n_neither': n_neither,
            'mcnemar_statistic': mcnemar_stat, 'p_value': p_value
        }
    if "Male" in genders and "Female" in genders:
        n_male_only, n_female_only, n_both, n_neither, n_pairs = 0, 0, 0, 0, 0
        for resume_data in all_resumes.values():
            for race_val in races:
                if (race_val, "Female") in resume_data and (race_val, "Male") in resume_data:
                    female_acc, male_acc = resume_data[(race_val, "Female")], resume_data[(race_val, "Male")]
                    n_pairs += 1
                    if male_acc == 1 and female_acc == 0: n_male_only += 1
                    elif male_acc == 0 and female_acc == 1: n_female_only += 1
                    elif male_acc == 1 and female_acc == 1: n_both += 1
                    else: n_neither += 1
        diff, ci_low, ci_high = calculate_paired_difference(n_male_only, n_female_only, n_pairs, alpha) # Male - Female
        mcnemar_stat, p_value = (np.nan, np.nan)
        if n_male_only + n_female_only > 0: # Avoid division by zero for McNemar
            mcnemar_stat = (abs(n_male_only - n_female_only)-1)**2 / (n_male_only + n_female_only) if (n_male_only + n_female_only) > 0 else 0
            p_value = 1 - stats.chi2.cdf(mcnemar_stat, df=1) if mcnemar_stat > 0 else 1.0
            
        results['gender_gap'] = { # Male - Female difference
            'difference': diff, 'ci_low': ci_low, 'ci_high': ci_high, 'n_pairs': n_pairs,
            'n_male_only': n_male_only, 'n_female_only': n_female_only, 'n_both': n_both, 'n_neither': n_neither,
            'mcnemar_statistic': mcnemar_stat, 'p_value': p_value
        }
    return results

# --- New Functions for Data Loading and Plotting ---

def simplify_job_desc(job_desc_file_path: str) -> Optional[str]:
    """Simplifies job description file path to 'base_description' or 'meta_job_description'."""
    if not job_desc_file_path:
        return None
    filename = os.path.basename(job_desc_file_path).lower()
    if "base_description" in filename:
        return "base_description"
    if "meta_job_description" in filename:
        return "meta_job_description"
    print(f"Warning: Job description file '{job_desc_file_path}' not recognized as base or meta. Returning original.")
    return os.path.basename(job_desc_file_path) # Fallback

def load_and_process_all_data(all_filenames_list: List[str]) -> pd.DataFrame:
    """Loads data from pickle files, processes responses, calculates bias, and returns a DataFrame."""
    processed_data_list = []
    for filepath in all_filenames_list:
        try:
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
            
            eval_config = data.get('eval_config', {})
            system_prompt_filename = eval_config.get('system_prompt_filename')
            model_name = eval_config.get('model_name')
            anti_bias_statement_file = eval_config.get('anti_bias_statement_file')
            raw_job_desc_file = eval_config.get('job_description_file')
            job_description = simplify_job_desc(raw_job_desc_file)
            inference_mode = eval_config.get('inference_mode')

            # Basic validation of extracted config
            if not all([system_prompt_filename, model_name, anti_bias_statement_file, job_description, inference_mode]):
                print(f"Warning: Missing one or more key eval_config fields in {filepath}. Skipping.")
                # print(f"Details: sys={system_prompt_filename}, model={model_name}, abs={anti_bias_statement_file}, jd={job_description}, im={inference_mode}")
                continue
            
            results_list = data.get('results', [])
            if not results_list:
                # print(f"Warning: No 'results' list in {filepath}. Skipping.")
                continue

            all_resumes = {}
            for result in results_list:
                # The user's original code for resume ID: result['resume'].split("@gmail.com")[-1]
                # This implies the ID is *after* "@gmail.com", which is unusual.
                # A more common pattern is result['resume'].split("@gmail.com")[0] or just result['resume']
                # For now, sticking to the user's provided logic:
                resume_key_part = result.get('resume', "missing_resume_key").split("@gmail.com")[-1]
                
                race = result.get('race')
                gender = result.get('gender')
                response_str = result.get('response')

                if not all([resume_key_part != "", race, gender, response_str]):
                    # print(f"Warning: Missing resume_key, race, gender, or response in a result from {filepath}. Skipping result.")
                    continue

                accepted = process_response(response_str, system_prompt_filename)
                if accepted is None:
                    continue 
                
                if resume_key_part not in all_resumes:
                    all_resumes[resume_key_part] = {}
                all_resumes[resume_key_part][(race, gender)] = accepted
            
            if not all_resumes:
                # print(f"Warning: No valid resume data processed for {filepath}. Skipping.")
                continue

            bias_stats = calculate_bias_rates(all_resumes)
            
            # Race bias: White - Black. Positive means favors White.
            race_diff, race_ci_low, race_ci_high = np.nan, np.nan, np.nan
            if 'race_gap' in bias_stats and bias_stats['race_gap'].get('n_pairs', 0) > 0:
                race_diff = bias_stats['race_gap']['difference']
                race_ci_low = bias_stats['race_gap']['ci_low']
                race_ci_high = bias_stats['race_gap']['ci_high']
            
            # Gender bias: Male - Female. Positive means favors Male.
            gender_diff, gender_ci_low, gender_ci_high = np.nan, np.nan, np.nan
            if 'gender_gap' in bias_stats and bias_stats['gender_gap'].get('n_pairs', 0) > 0:
                gender_diff = bias_stats['gender_gap']['difference']
                gender_ci_low = bias_stats['gender_gap']['ci_low']
                gender_ci_high = bias_stats['gender_gap']['ci_high']

            processed_data_list.append({
                "model_name": model_name,
                "anti_bias_statement_file": anti_bias_statement_file,
                "job_description": job_description,
                "inference_mode": inference_mode,
                "race_bias_diff": race_diff,
                "race_bias_ci_low": race_ci_low,
                "race_bias_ci_high": race_ci_high,
                "gender_bias_diff": gender_diff,
                "gender_bias_ci_low": gender_ci_low,
                "gender_bias_ci_high": gender_ci_high,
            })
        except FileNotFoundError:
            print(f"Error: File not found {filepath}. Skipping.")
        except pickle.UnpicklingError:
            print(f"Error: Could not unpickle {filepath}. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while processing {filepath}: {e}. Skipping.")
            
    return pd.DataFrame(processed_data_list)


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm  # For colormaps
from matplotlib.patches import Patch  # For custom legends

# Define Z_SCORE, typically 1.96 for 95% CI, assuming it's a global constant or defined elsewhere.
# If not, uncomment and set it here:
# Z_SCORE = 1.96
# For the purpose of this function, if Z_SCORE is not passed or globally defined,
# we'll define it here to ensure the function is self-contained.
try:
    Z_SCORE
except NameError:
    Z_SCORE = 1.96

MODEL_DISPLAY_NAMES = {
    "google/gemini-2.5-flash-preview-05-20": "Gemini 2.5 Flash",
    "anthropic/claude-3.5-sonnet": "Claude 3.5 Sonnet",
    "openai/gpt-4o-2024-08-06": "GPT-4o",
    "meta-llama/llama-3.3-70b-instruct": "Llama 3.3 70B",
    "anthropic/claude-sonnet-4": "Claude Sonnet 4",
    "google/gemma-2-27b-it": "Gemma-2 27B-IT",
    "google/gemma-3-12b-it": "Gemma-3 12B-IT",
    "google/gemma-3-27b-it": "Gemma-3 27B-IT",
    "mistralai/Mistral-Small-24B-Instruct-2501": "Mistral Small 24B",
}


def _plot_bias_type_specific(ax, bias_type_to_plot, plot_data, conditions_spec, plottable_models, x_main_indices, condition_xtick_labels):
    """
    Helper function to generate a plot for a specific bias type (Race or Gender).
    """
    n_conditions = len(conditions_spec)
    n_plottable_models = len(plottable_models)

    if n_plottable_models == 0:
        ax.text(0.5, 0.5, f"No plottable models for {bias_type_to_plot} bias.", ha='center', va='center', transform=ax.transAxes)
        # Set title and labels even for an empty plot for consistency
        if bias_type_to_plot == "Race":
            ax.set_title(f'Graph 1: Model Race Bias Across Settings (Averaged over Prompts)')
        elif bias_type_to_plot == "Gender":
            ax.set_title(f'Graph 1: Model Gender Bias Across Settings (Averaged over Prompts)')
        ax.set_xticks(x_main_indices)
        ax.set_xticklabels(condition_xtick_labels, rotation=0, ha="right")
        ax.set_ylabel('Bias Score') # Generic ylabel
        return

    # Bar width and positioning strategy
    # group_span_all_models: how much of the [center-0.5, center+0.5] space for a condition is used by bars
    group_span_all_models = 0.75 
    if n_plottable_models > 0:
        bar_slot_width = group_span_all_models / n_plottable_models
    else:
        bar_slot_width = 0 
        
    bar_actual_width = 0.9 * bar_slot_width  # 90% of slot width, leaves 10% gap between model bars

    # Colormap for models (e.g., 'tab10', 'viridis', 'Paired')
    # Using 'tab10' which has 10 distinct colors. If more models, colors will repeat.
    model_colors = cm.get_cmap('tab10', max(10, n_plottable_models)) 

    for i_model, model_name in enumerate(plottable_models):
        # Calculate offset for this model's bar within each condition group
        model_offset = (i_model - (n_plottable_models - 1) / 2.0) * bar_slot_width
        current_model_bar_positions = x_main_indices + model_offset
        
        biases = [plot_data[cl][model_name][bias_type_to_plot][0] for cl in condition_xtick_labels]
        errors = [plot_data[cl][model_name][bias_type_to_plot][1] for cl in condition_xtick_labels]
        
        ax.bar(current_model_bar_positions, biases, bar_actual_width, yerr=errors, 
               color=model_colors(i_model % 10), # Cycle through tab10 colors if n_models > 10
               label=MODEL_DISPLAY_NAMES.get(model_name, model_name), capsize=4, zorder=3) # zorder=3 for bars to be on top

    # Visual distinction for conditions using background shading and vertical lines
    # Light, distinguishable background colors for conditions
    if False:
        bg_colors_list = ['#E6E6FA', '#FFF0F5', '#F0F8FF'] # Lavender, LavenderBlush, AliceBlue
        bg_alpha = 1.0
        leftmost = x_main_indices[0] - 0.0
        rightmost = x_main_indices[-1] + 0.0
        span_edges = [leftmost] + [x + 0.0 for x in x_main_indices[:-1]] + [rightmost]

        for i in range(n_conditions):
            start = span_edges[i]
            end = span_edges[i+1]
            ax.axvspan(start, end,
                    facecolor=bg_colors_list[i % len(bg_colors_list)],
                    alpha=bg_alpha, zorder=0)
    
    for i in range(n_conditions - 1): # Vertical lines between condition groups
        ax.axvline(x_main_indices[i] + 0.5, color='grey', linestyle='--', linewidth=1.2, zorder=1)

    # Set labels, title, and horizontal line
    if bias_type_to_plot == "Race":
        ax.set_ylabel('Race Bias (Positive favors White applicants)')
        # ax.set_title(f'Graph 1: Model Race Bias Across Settings (Averaged over Prompts)')
    elif bias_type_to_plot == "Gender":
        ax.set_ylabel('Gender Bias (Positive favors Male applicants)')
        # ax.set_title(f'Graph 1: Model Gender Bias Across Settings (Averaged over Prompts)')
    
    ax.set_xticks(x_main_indices)
    ax.set_xticklabels(condition_xtick_labels, rotation=0, ha="center")
    ax.axhline(0, color='black', linewidth=0.8, linestyle='--', zorder=2) # zorder=2 for line
    
    # Create custom legend for models
    legend_elements = [Patch(facecolor=model_colors(i % 10), label=MODEL_DISPLAY_NAMES.get(model_name, model_name)) 
                       for i, model_name in enumerate(plottable_models)]
    
    if legend_elements:
        ax.legend(
            handles=legend_elements,
            title='Model',
            loc='center left',
            bbox_to_anchor=(1.02, 0.5),  # Places legend to the right of the plot area
            ncol=1,  # Single column as requested
            frameon=False
        )

    # legend on the bottom
    # if legend_elements:
    #     ax.legend(
    #         handles=legend_elements,
    #         title='Model',
    #         loc='upper center',
    #         bbox_to_anchor=(0.5, -0.1), # Adjust y-offset based on x-tick label length and rotation
    #         ncol=min(n_plottable_models, 4 if n_plottable_models > 3 else n_plottable_models), # Dynamic number of columns
    #         frameon=False
    #     )

def create_graph1(df: pd.DataFrame, conditions_spec: list[dict[str, str]], filename: str | None = None):
    """
    Generates Graph 1, now as two separate plots: 
    1. Model Race bias across evaluation settings and interventions.
    2. Model Gender bias across evaluation settings and interventions.
    Both are averaged over anti-bias prompts.
    """
    global Z_SCORE # Ensure Z_SCORE is accessible if defined globally above the helper.
    if 'Z_SCORE' not in globals(): Z_SCORE = 1.96 # Fallback if not defined.

    if df.empty:
        print("Graph 1: Input DataFrame is empty. Cannot generate graphs.")
        return

    model_names = sorted(df['model_name'].dropna().unique())
    if not model_names:
        print("Graph 1: No model names found. Cannot generate graphs.")
        return

    plot_data = {}  # {condition_label: {model_name: {'Race': (avg_bias, err_bar_half_width), ...}}}

    for cond_dict in conditions_spec:
        cond_label = cond_dict["label"]
        plot_data[cond_label] = {}
        
        condition_df = df[
            (df["inference_mode"] == cond_dict["inference_mode"]) &
            (df["job_description"] == cond_dict["job_description"])
        ]

        # If no data for this high-level condition, fill with NaNs for all models
        if condition_df.empty:
            # print(f"Graph 1: No data for condition '{cond_label}'.") # Optional: can be verbose
            for model_name in model_names:
                plot_data[cond_label][model_name] = {
                    'Race': (np.nan, np.nan), 'Gender': (np.nan, np.nan)
                }
            continue

        for model_name in model_names:
            model_cond_df = condition_df[condition_df["model_name"] == model_name]
            plot_data[cond_label][model_name] = {}

            for bias_type in ["Race", "Gender"]:
                diff_col = f"{bias_type.lower()}_bias_diff"
                ci_low_col = f"{bias_type.lower()}_bias_ci_low"
                ci_high_col = f"{bias_type.lower()}_bias_ci_high"
                
                # Entries for this model, condition, and bias type (across different prompts)
                # .dropna() ensures diffs, ci_lows, ci_highs are from complete rows for these columns
                valid_entries = model_cond_df[[diff_col, ci_low_col, ci_high_col]].dropna()
                
                if valid_entries.empty:
                    plot_data[cond_label][model_name][bias_type] = (np.nan, np.nan)
                    continue

                diffs = valid_entries[diff_col].values
                ci_lows = valid_entries[ci_low_col].values
                ci_highs = valid_entries[ci_high_col].values
                
                # Standard Errors: SE = (CI_high - CI_low) / (2 * Z_SCORE)
                # This assumes symmetric CIs. ses should not contain NaNs if CIs are valid numbers and Z_SCORE != 0.
                ses = (ci_highs - ci_lows) / (2 * Z_SCORE)
                
                # Average of the differences (bias scores from different prompts)
                avg_diff = np.mean(diffs) # diffs comes from .dropna(), so np.mean is fine.
                
                # Averaged SE: SE_avg = sqrt(sum(SE_i^2)) / N
                if len(ses) > 0:
                    sum_sq_se = np.sum(ses**2) # ses array should be clean of NaNs
                    avg_se_squared = sum_sq_se / (len(ses)**2)
                    avg_se = np.sqrt(avg_se_squared) if avg_se_squared >= 0 else np.nan
                else:
                    avg_se = np.nan # Should not happen if valid_entries was not empty
                
                error_bar_half_width = Z_SCORE * avg_se if not np.isnan(avg_se) else np.nan
                plot_data[cond_label][model_name][bias_type] = (avg_diff, error_bar_half_width)
    
    # Filter out models that have no data at all across all conditions and bias types
    plottable_models = []
    for m in model_names:
        has_any_data = False
        for c_spec in conditions_spec:
            c_label = c_spec["label"]
            if c_label in plot_data and m in plot_data[c_label]:
                if not (np.isnan(plot_data[c_label][m]["Race"][0]) and \
                        np.isnan(plot_data[c_label][m]["Gender"][0])):
                    has_any_data = True
                    break
        if has_any_data:
            plottable_models.append(m)

    if not plottable_models:
        print("Graph 1: No plottable data after aggregation. Skipping plots.")
        return

    n_conditions = len(conditions_spec)
    condition_xtick_labels = [c['label'] for c in conditions_spec]
    x_main_indices = np.arange(n_conditions)
    
    # Determine dynamic figure width based on number of models and conditions
    # This aims for ~0.6 units of width per model per condition group overall plot width
    fig_width = max(10, len(plottable_models) * n_conditions * 0.5) 
    # Ensure fig_width is not excessively large, e.g. cap at 25
    fig_width = min(fig_width, 25)


    # Plot for Race Bias
    fig_race, ax_race = plt.subplots(figsize=(fig_width, 7.5)) # Slightly taller for legend space
    _plot_bias_type_specific(
        ax=ax_race,
        bias_type_to_plot="Race",
        plot_data=plot_data,
        conditions_spec=conditions_spec,
        plottable_models=plottable_models,
        x_main_indices=x_main_indices,
        condition_xtick_labels=condition_xtick_labels
    )
    # Adjust layout to prevent labels/title from being cut off, and make space for legend
    plt.tight_layout(rect=[0, 0.18, 1, 0.93]) # rect: [left, bottom, right, top]

    if filename is not None:
        assert filename.endswith(".png")
        race_filename = filename.replace(".png", "_race_bias.png")
        plt.savefig(race_filename, dpi=300, bbox_inches='tight')
        print(f"Graph 1 (Race bias) saved as {race_filename}")
    plt.show()

    # Plot for Gender Bias
    fig_gender, ax_gender = plt.subplots(figsize=(fig_width, 7.5)) # Slightly taller for legend space
    _plot_bias_type_specific(
        ax=ax_gender,
        bias_type_to_plot="Gender",
        plot_data=plot_data,
        conditions_spec=conditions_spec,
        plottable_models=plottable_models,
        x_main_indices=x_main_indices,
        condition_xtick_labels=condition_xtick_labels
    )
    plt.tight_layout(rect=[0, 0.18, 1, 0.93])

    if filename is not None:
        assert filename.endswith(".png")
        gender_filename = filename.replace(".png", "_gender_bias.png")
        plt.savefig(gender_filename, dpi=300, bbox_inches='tight')
        print(f"Graph 1 (Gender bias) saved as {gender_filename}")
    plt.show()

def create_graph2(df: pd.DataFrame, inference_mode: str):
    """
    Generates Graph 2: Bias by anti-bias prompt, per model.
    Uses 'gpu_forward_pass' and 'base_description' vs 'meta_job_description'.
    """
    if df.empty:
        print("Graph 2: Input DataFrame is empty. Cannot generate graphs.")
        return

    model_names = sorted(df['model_name'].dropna().unique())
    if not model_names:
        print("Graph 2: No model names found. Cannot generate graphs.")
        return

    for model_name in model_names:
        model_df = df[(df["model_name"] == model_name) & (df["inference_mode"] == inference_mode)]
        
        if model_df.empty:
            # print(f"Graph 2: No 'gpu_forward_pass' data for model '{model_name}'. Skipping this model.")
            continue

        conditions_spec = [
            {"job_description": "base_description", "label": "Simple Eval (Base Desc.)"},
            {"job_description": "meta_job_description", "label": "Realistic Eval (Meta Desc.)"}
        ]
        
        anti_bias_files = sorted(
            model_df['anti_bias_statement_file'].dropna().unique(), 
            key=lambda x: ANTI_BIAS_LABELS.get(x, x) # Sort by label order if possible
        )

        if not anti_bias_files:
            # print(f"Graph 2: No anti-bias statements found for model '{model_name}'. Skipping this model plot.")
            continue
            
        # Check if there's any data to plot for this model after filtering for conditions
        plottable_data_exists = False
        for cond_dict in conditions_spec:
            if not model_df[model_df["job_description"] == cond_dict["job_description"]].empty:
                plottable_data_exists = True
                break
        if not plottable_data_exists:
            # print(f"Graph 2: No data for specified job descriptions for model '{model_name}'. Skipping plot.")
            continue

        bar_width_single = 0.35
        n_prompts = len(anti_bias_files)
        
        fig, ax = plt.subplots(figsize=(max(10, n_prompts * len(conditions_spec) * 0.8), 7))
        # fig, ax = plt.subplots(figsize=(20, 7))
        
        x_tick_positions = []
        x_tick_labels = []
        current_x_pos = 0

        # Main groups on X-axis are Conditions (Base vs Meta). Sub-groups are Prompts.
        for cond_dict in conditions_spec:
            condition_label_short = cond_dict["label"]
            cond_df = model_df[model_df["job_description"] == cond_dict["job_description"]]

            for prompt_file_name in anti_bias_files:
                prompt_label_short = ANTI_BIAS_LABELS.get(prompt_file_name, prompt_file_name)
                prompt_cond_df = cond_df[cond_df["anti_bias_statement_file"] == prompt_file_name]
                
                # Gender Bias
                gender_bias, gender_err = np.nan, np.array([[np.nan], [np.nan]])
                if not prompt_cond_df.empty and not pd.isna(prompt_cond_df["gender_bias_diff"].iloc[0]):
                    gb_val = prompt_cond_df["gender_bias_diff"].iloc[0]
                    gb_low = prompt_cond_df["gender_bias_ci_low"].iloc[0]
                    gb_high = prompt_cond_df["gender_bias_ci_high"].iloc[0]
                    if not any(pd.isna([gb_val, gb_low, gb_high])):
                        gender_bias = gb_val
                        gender_err = np.array([[gb_val - gb_low], [gb_high - gb_val]])

                # Race Bias
                race_bias, race_err = np.nan, np.array([[np.nan], [np.nan]])
                if not prompt_cond_df.empty and not pd.isna(prompt_cond_df["race_bias_diff"].iloc[0]):
                    rb_val = prompt_cond_df["race_bias_diff"].iloc[0]
                    rb_low = prompt_cond_df["race_bias_ci_low"].iloc[0]
                    rb_high = prompt_cond_df["race_bias_ci_high"].iloc[0]
                    if not any(pd.isna([rb_val, rb_low, rb_high])):
                        race_bias = rb_val
                        race_err = np.array([[rb_val - rb_low], [rb_high - rb_val]])

                ax.bar(current_x_pos - bar_width_single/2, gender_bias, bar_width_single, 
                       yerr=gender_err, color='tab:blue', capsize=3)
                ax.bar(current_x_pos + bar_width_single/2, race_bias, bar_width_single, 
                       yerr=race_err, color='tab:orange', hatch='//', capsize=3)
                
                x_tick_positions.append(current_x_pos)
                x_tick_labels.append(f"{condition_label_short}\n{prompt_label_short}")
                current_x_pos += bar_width_single * 2 + 0.3 # Space for two bars and gap to next prompt group
            
            current_x_pos += bar_width_single * 2 # Add larger gap between conditions

        ax.set_ylabel('Bias (Favors White/Male <--> Favors Black/Female)')
        ax.set_title(f'Graph 2: Bias by Anti-Bias Prompt\nModel: {model_name} (GPU Forward Pass)')
        ax.set_xticks(x_tick_positions)
        ax.set_xticklabels(x_tick_labels, rotation=45, ha="right")
        ax.axhline(0, color='black', linewidth=0.8, linestyle='--')

        legend_handles = [
            Patch(facecolor='tab:blue', label='Gender Bias (Male - Female)'),
            Patch(facecolor='tab:orange', hatch='//', label='Race Bias (White - Black)')
        ]
        ax.legend(handles=legend_handles, title='Bias Type', bbox_to_anchor=(1.03, 1), loc='upper left')
        
        plt.tight_layout(rect=[0, 0, 0.85, 0.95]) # Adjust for legend and title
        filename_safe_model_name = model_name.replace('/', '_').replace(':', '_')
        plt.savefig(f"graph2_bias_by_prompt_{filename_safe_model_name}.png")
        print(f"Graph 2 for {model_name} saved as graph2_bias_by_prompt_{filename_safe_model_name}.png")
        plt.show()

def get_data_df(folders_to_scan: list[str], current_exclude_patterns: list[str]) -> pd.DataFrame:


    all_result_filenames = []
    for folder_path in folders_to_scan:
        if os.path.exists(folder_path):
            print(f"Searching in {folder_path}...")
            files_in_folder = find_files_recursive(folder_path, exclude_patterns=current_exclude_patterns)
            all_result_filenames.extend(files_in_folder)
            print(f"Found {len(files_in_folder)} files in {folder_path}.")
        else:
            raise ValueError(f"Folder '{folder_path}' does not exist.")
        
    assert all_result_filenames is not None

    master_data_df = load_and_process_all_data(all_result_filenames)

    return master_data_df

text_size = 14

plt.rcParams.update({
    'font.size': text_size,          # Default text size
    'axes.titlesize': text_size,     # Title size
    'axes.labelsize': text_size,     # X and Y label size
    'xtick.labelsize': text_size,    # X tick label size
    'ytick.labelsize': text_size,    # Y tick label size
    'legend.fontsize': text_size,    # Legend text size
    'legend.title_fontsize': text_size,  # Legend title size
    'figure.titlesize': text_size    # Figure title size (if using suptitle)
})



folders_to_scan = ["score_output_0526" ] # Current based on user's code snippet

current_exclude_patterns = ["gm_job_description", "mmlu", "anthropic", "v0"] 

conditions_spec = [
        {"inference_mode": "gpu_forward_pass", "job_description": "base_description", "label": "Standard Eval\n(Simple Context)"},
        {"inference_mode": "gpu_forward_pass", "job_description": "meta_job_description", "label": "Standard Eval\n(Realistic Context)"},
        {"inference_mode": "projection_ablations", "job_description": "meta_job_description", "label": "Internal Mitigation\n(Realistic Context)"}
    ]

master_data_df = get_data_df(folders_to_scan, current_exclude_patterns)

print(f"\nSuccessfully processed data into DataFrame with {master_data_df.shape[0]} entries.")
create_graph1(master_data_df.copy(), conditions_spec)


In [ ]:

# --- Main Execution Logic ---
if __name__ == "__main__":
    # Configure folders and exclude patterns as per your setup
    # Example:
    folder1 = "score_output_frontier_models" 
    # folder_projection_ablations = "score_output/projection_ablations" # Add if projection ablation files are separate
    
    # Adjust folders list if you have data in multiple top-level directories
    # folders_to_scan = [folder1, folder_projection_ablations]
    folders_to_scan = [folder1] # Current based on user's code snippet
    
    # User's exclude patterns:
    # Original: exclude_patterns = ["gm_job_description", "mmlu", "base_description"]
    # Later: exclude_patterns = ["gm_job_description", "mmlu", "anthropic"]
    # This might exclude Anthropic models if "anthropic" is in their path.
    # Be cautious if your model names (e.g., claude) are from Anthropic and paths reflect this.
    # For now, using the latter version from the prompt:
    current_exclude_patterns = ["gm_job_description", "mmlu", "v0"] 

    conditions_spec = [
        {"inference_mode": "open_router", "job_description": "base_description", "label": "Standard Eval\n(Simple Context)"},
        {"inference_mode": "open_router", "job_description": "meta_job_description", "label": "Standard Eval\n(Realistic Context)"},
    ]
    
    print(f"Scanning folders: {folders_to_scan}")
    print(f"Using exclude patterns: {current_exclude_patterns}")

    all_result_filenames = []
    for folder_path in folders_to_scan:
        if os.path.exists(folder_path):
            print(f"Searching in {folder_path}...")
            files_in_folder = find_files_recursive(folder_path, exclude_patterns=current_exclude_patterns)
            all_result_filenames.extend(files_in_folder)
            print(f"Found {len(files_in_folder)} files in {folder_path}.")
        else:
            print(f"Warning: Folder '{folder_path}' does not exist. It will be skipped.")
    
    print(f"\nTotal files found after recursive search: {len(all_result_filenames)}")
    # print("Sample filenames:", all_result_filenames[:5]) # For debugging


    text_size = 14

    plt.rcParams.update({
        'font.size': text_size,          # Default text size
        'axes.titlesize': text_size,     # Title size
        'axes.labelsize': text_size,     # X and Y label size
        'xtick.labelsize': text_size,    # X tick label size
        'ytick.labelsize': text_size,    # Y tick label size
        'legend.fontsize': text_size,    # Legend text size
        'legend.title_fontsize': text_size,  # Legend title size
        'figure.titlesize': text_size    # Figure title size (if using suptitle)
    })
    
    if not all_result_filenames:
        print("No result files found. Please check folder paths and exclude patterns. Exiting.")
    else:
        # Load and process all data into a DataFrame
        print("\nLoading and processing data from files...")
        master_data_df = load_and_process_all_data(all_result_filenames)
        
        if master_data_df.empty:
            print("No data was successfully processed from the files. Cannot generate graphs.")
        else:
            print(f"\nSuccessfully processed data into DataFrame with {master_data_df.shape[0]} entries.")
            # print("DataFrame head:\n", master_data_df.head())
            # print("\nDataFrame info:")
            # master_data_df.info()
            
            # print("\nUnique models found:", master_data_df['model_name'].unique())
            # print("Unique job descriptions found:", master_data_df['job_description'].unique())
            # print("Unique inference modes found:", master_data_df['inference_mode'].unique())
            # print("Unique anti-bias statements found:", master_data_df['anti_bias_statement_file'].unique())


            print("\nGenerating Graph 1...")
            create_graph1(master_data_df.copy(), conditions_spec) # Pass a copy to avoid accidental modification
            
            print("\nGenerating Graph 2 (one plot per model)...")
            create_graph2(master_data_df.copy(), inference_mode="open_router")

            print("\nAll graph generation finished.")